In [1]:
from binance.client import Client
import pandas as pd

API_KEY = '452dc5d98173445e670b58738c260b472776498c15d0fa4bdce4a6f84833217d'
API_SECRET = ''
client = Client(API_KEY, API_SECRET)

ModuleNotFoundError: No module named 'binance'

In [1]:
import pandas as pd
date_from = "2023-01-01"
date_to = "2023-03-31"
dates = pd.date_range(date_from, date_to).astype(str).str.replace("-", "")

In [2]:
import requests
from os import path
BASE_URL = "https://s3-eu-west-1.amazonaws.com/public.bitmex.com/data/trade/%s"

def download_trade_file(filename, output_folder):
    print(f"Downloading {filename} file")
    url = BASE_URL % filename
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"Cannot download the {filename} file. Status code: {resp.status_code}")
        return
    with open(path.join(output_folder, filename), "wb") as f:
        f.write(resp.content)
    print(f"{filename} downloaded")


In [3]:
output_folder = "data/bitmex"
# for date in dates:
#     filename = date + ".csv.gz"
#     download_trade_file(filename, output_folder)

In [4]:
import glob
filepaths = glob.glob(path.join(output_folder, "*.csv.gz"))
filepaths = sorted(filepaths)
filepaths

['data/bitmex/20230201.csv.gz',
 'data/bitmex/20230202.csv.gz',
 'data/bitmex/20230203.csv.gz',
 'data/bitmex/20230204.csv.gz',
 'data/bitmex/20230205.csv.gz',
 'data/bitmex/20230206.csv.gz',
 'data/bitmex/20230207.csv.gz',
 'data/bitmex/20230208.csv.gz',
 'data/bitmex/20230209.csv.gz',
 'data/bitmex/20230210.csv.gz',
 'data/bitmex/20230211.csv.gz',
 'data/bitmex/20230212.csv.gz',
 'data/bitmex/20230213.csv.gz',
 'data/bitmex/20230214.csv.gz',
 'data/bitmex/20230215.csv.gz',
 'data/bitmex/20230216.csv.gz',
 'data/bitmex/20230217.csv.gz',
 'data/bitmex/20230218.csv.gz',
 'data/bitmex/20230219.csv.gz',
 'data/bitmex/20230220.csv.gz',
 'data/bitmex/20230221.csv.gz',
 'data/bitmex/20230222.csv.gz',
 'data/bitmex/20230223.csv.gz',
 'data/bitmex/20230224.csv.gz',
 'data/bitmex/20230225.csv.gz',
 'data/bitmex/20230226.csv.gz',
 'data/bitmex/20230227.csv.gz',
 'data/bitmex/20230228.csv.gz',
 'data/bitmex/20230301.csv.gz',
 'data/bitmex/20230302.csv.gz',
 'data/bitmex/20230303.csv.gz',
 'data/b

In [5]:
df_list = []
for filepath in filepaths:
    print(f"Reading {filepath}")
    df_ = pd.read_csv(filepath)
    df_ = df_[df_.symbol == "XBTUSD"]
    print(f"Read {df_.shape[0]} rows")
    df_list.append(df_)
df = pd.concat(df_list)
df_list = None
df_ = None

Reading data/bitmex/20230201.csv.gz
Read 100624 rows
Reading data/bitmex/20230202.csv.gz
Read 109346 rows
Reading data/bitmex/20230203.csv.gz
Read 87251 rows
Reading data/bitmex/20230204.csv.gz
Read 41117 rows
Reading data/bitmex/20230205.csv.gz
Read 65611 rows
Reading data/bitmex/20230206.csv.gz
Read 78951 rows
Reading data/bitmex/20230207.csv.gz
Read 89560 rows
Reading data/bitmex/20230208.csv.gz
Read 70514 rows
Reading data/bitmex/20230209.csv.gz
Read 125656 rows
Reading data/bitmex/20230210.csv.gz
Read 72277 rows
Reading data/bitmex/20230211.csv.gz
Read 34197 rows
Reading data/bitmex/20230212.csv.gz
Read 48529 rows
Reading data/bitmex/20230213.csv.gz
Read 81154 rows
Reading data/bitmex/20230214.csv.gz
Read 93831 rows
Reading data/bitmex/20230215.csv.gz
Read 115725 rows
Reading data/bitmex/20230216.csv.gz
Read 145247 rows
Reading data/bitmex/20230217.csv.gz
Read 124235 rows
Reading data/bitmex/20230218.csv.gz
Read 49718 rows
Reading data/bitmex/20230219.csv.gz
Read 98133 rows
Readin

In [6]:
df.loc[:, "Datetime"] = pd.to_datetime(df.timestamp.str.replace("D", "T"))
df.drop("timestamp", axis=1, inplace=True)

In [7]:
df_test = df.copy()

In [8]:
df = df.groupby(pd.Grouper(key="Datetime", freq="15Min")).agg(
    {"price": ["first", "max", "min", "last"], "foreignNotional": "sum"}
)
df.columns = ["Open", "High", "Low", "Close", "Volume"]
df.loc[:, "OpenInterest"] = 0.0 # required by backtrader
df = df[df.Close.notnull()]
df.reset_index(inplace=True)

In [9]:
df

,Datetime,Open,High,Low,Close,Volume,OpenInterest
0,2023-02-01 00:00:00,23114.0,23153.0,23029.5,23079.5,5107700.0,0.0
1,2023-02-01 00:15:00,23079.5,23079.5,23018.0,23028.0,2323600.0,0.0
2,2023-02-01 00:30:00,23028.0,23083.0,22986.5,23067.5,4325000.0,0.0
3,2023-02-01 00:45:00,23070.0,23101.5,23067.0,23070.5,1364700.0,0.0
4,2023-02-01 01:00:00,23070.0,23106.5,23070.0,23106.5,1443600.0,0.0
...,...,...,...,...,...,...,...
5655,2023-03-31 22:45:00,28565.5,28590.5,28533.0,28564.5,1324400.0,0.0
5656,2023-03-31 23:00:00,28564.0,28585.0,28517.5,28517.5,2006100.0,0.0
5657,2023-03-31 23:15:00,28517.5,28544.5,28507.5,28543.0,1313700.0,0.0
5658,2023-03-31 23:30:00,28542.0,28545.0,28462.5,28467.0,1645500.0,0.0


In [10]:
# def binanceDataFrame(klines):
#     df = pd.DataFrame(klines,dtype=float, columns = ('Open Time',
#                                                                     'open',
#                                                                     'high',
#                                                                     'low',
#                                                                     'close',
#                                                                     'volume',
#                                                                     'Close time',
#                                                                     'Quote asset volume',
#                                                                     'Number of trades',
#                                                                     'Taker buy base asset volume',
#                                                                     'Taker buy quote asset volume',
#                                                                     'Ignore'))
#
#     df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
#     df = df.set_index('Open Time')
#
#     return df
#
# new_date_load = False
# if new_date_load:
#     candlesticks = client.get_historical_klines("XRPUSDT", Client.KLINE_INTERVAL_5MINUTE, '2 years')#"31 Aug, 2021", "1 Sep, 2021")
#     df = binanceDataFrame(candlesticks)
# else:
#
#     df = pd.read_csv("xrp_sincejan5min.csv")
#     df['Open Time'] = pd.to_datetime(df['Open Time'])
#     df = df.set_index('Open Time')



In [11]:
# df.to_csv("xrp_2year.csv")

In [12]:
import backtrader as bt
import datetime as dt

import backtrader as bt
import backtrader.indicators as btind

# #Add data feed to Cerebro
data = bt.feeds.PandasData(dataname=df)


In [13]:
from sklearn.linear_model import LinearRegression


In [14]:
def printTradeAnalysis(cerebro, analyzers):
    print('Backtesting Results')
    if hasattr(analyzers, 'ta'):
        ta = analyzers.ta.get_analysis()

        openTotal         = ta.total.open          
        closedTotal       = ta.total.closed        
        wonTotal          = ta.won.total           
        lostTotal         = ta.lost.total          

        streakWonLongest  = ta.streak.won.longest  
        streakLostLongest = ta.streak.lost.longest 

        pnlNetTotal       = ta.pnl.net.total       
        pnlNetAverage     = ta.pnl.net.average     

        print('Open Positions', openTotal  )
        print('Closed Trades',  closedTotal)
        print('Winning Trades', wonTotal   )
        print('Loosing Trades', lostTotal  )
       

        print('Longest Winning Streak',   streakWonLongest )
        print('Longest Loosing Streak',   streakLostLongest)
        print('Strike Rate (Win/closed)', (wonTotal / closedTotal) * 100 if wonTotal and closedTotal else 0)
        

#         print(format, 'Inital Portfolio Value', '${}'.format(100))
        print( 'Final Portfolio Value',  '${}'.format(cerebro.broker.getvalue()))
        print( 'Net P/L',                '${}'.format(round(pnlNetTotal,   2)) )
        print( 'P/L Average per trade',  '${}'.format(round(pnlNetAverage, 2)))
        print('\n')

    if hasattr(analyzers, 'drawdown'):
        print('Drawdown', '${}'.format(analyzers.drawdown.get_analysis()['drawdown']))
    if hasattr(analyzers, 'sharpe'):
        print( 'Sharpe Ratio:', analyzers.sharpe.get_analysis()['sharperatio'])
    if print(analyzers, 'vwr'):
        print( 'VRW', analyzers.vwr.get_analysis()['vwr'])
    if hasattr(analyzers, 'sqn'):
        print( 'SQN', analyzers.sqn.get_analysis()['sqn'])
    print('\n')

    print('Transactions')
    print( 'Date', 'Amount', 'Price', 'SID', 'Symbol', 'Value')
    for key, value in analyzers.txn.get_analysis().items():
        print( key.strftime("%Y/%m/%d %H:%M:%S"), value[0][0], value[0][1], value[0][2], value[0][3], value[0][4])


In [15]:
"""
Defines class / functions tools for strategies.
"""
import backtrader as bt



In [144]:

class firstStrategy(bt.Strategy):
    params = (
        ('rsi_period',14),
        ('fast_rsi_period',8),
        ('ema_period',13),
        ('stop_loss',1),
        ('ema1',20),
        ('ema2',50),
        ('ema3',200),

    )
    def __init__(self):
        self.startcash = self.broker.getvalue()
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.rsi_period)
        self.rsi_fast = bt.indicators.RSI(self.data.close, period=8)
        self.ema1 = bt.indicators.EMA(self.data.close, period=self.params.ema1) 
        self.ema2 = bt.indicators.EMA(self.data.close, period=self.params.ema2) 

        self.ema3 = bt.indicators.EMA(self.data.close, period=self.params.ema3) 


        self.o_li = list()

        # To keep track of pending orders
        self.order = None  
        self.stopOrder = None

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
#         dt = dt or self.datas[0].datetime.datetime(0)
# #         print('%s, %s' % (dt.isoformat(), txt))
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

#         self.log("OPERATION PROFIT, GROSS %.2f, NET %.2f" % (trade.pnl, trade.pnlcomm))
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # An active Buy/Sell order has been submitted/accepted - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash

        if order.status in [order.Completed]:
#             if order.isbuy():
#                 self.log(f'BUY EXECUTED, {order.executed.price:.5f}')
# #                 stop_price = order.executed.price * (1.0 - self.params.stop_loss)
# #                 self.sell(exectype=bt.Order.Stop, price=stop_price)
#             elif order.issell():
#                 self.log(f'SELL EXECUTED, {order.executed.price:.5f}')
#                 self.log
            self.bar_executed = len(self)

        
    def next(self):

        if not self.position:
            if self.rsi<30 and self.ema2>self.ema3:
                size = 1000/self.datas[0].close[0]

                self.order = self.buy(price=(self.datas[0].close[0]),exectype=bt.Order.Market,size=size)
#             if self.rsi>70 and self.order.Completed:
#                 self.order = self.sell(price=(self.datas[0].close[0]),exectype=bt.Order.Market,size=1000)

        else:
            if self.order and (self.data.close - self.order.price)!=0.0:
                change = (self.data.close - self.order.price)/self.order.price
                if change>0.04 or change<-0.02:
#                     print("selling at change:", change)
                    self.sell()

    def stop(self):
        pnl = round(self.broker.getvalue() - 2000,2)
        print('RSI Period: {} EMA1: {} EMA2 : {} EMA3: {} Final PnL: {}'.format(
            self.params.rsi_period, self.params.ema1,self.params.ema2,self.params.ema3,pnl))

In [145]:
class MyStrategy(bt.Strategy):


    params = (
        ('verbose', 1),
        ('maperiod', 14),
        ('quantity', .01),
        ('upper', 70),         # upper threshold
        ('lower', 30),         # lower threshold
        ('stopLoss', 0.01)          # stop loss %
    )
    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close, period=14)
        self.order = None
        self.size = None

        # Set the sizer to use the entire portfolio
        self.broker.setcommission(commission=0.0)
        # self.sizer = bt.sizers.PercentSizer(percents=100)

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')

    def next(self):

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.rsi < 25:

                # Keep track of the created order to avoid a 2nd order
                self.amount = (self.broker.getvalue()*100) / self.datas[0].close[0]
                self.order = self.buy()

        else:
            # Already in the market ... we might sell
            if self.rsi > 60:

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

                if self.params.verbose:
                    print('BOUGHT @price: {:.2f} {}'.format(order.executed.price, bt.num2date(order.executed.dt)))
                if self.params.stopLoss:
                    if self.params.stopLoss > 0:
                        stop_price = order.executed.price * (1 - self.params.stopLoss)
                        self.order_stopLoss = self.sell(exectype=bt.Order.Stop, price=stop_price)
                        if self.params.verbose:
                            print('  STOP @price: {:.2f}'.format(stop_price))
                    else:
                        # trailing stop specified % under executed price
                        self.order_stopLoss = self.sell(exectype=bt.Order.StopTrail, trailpercent=0-self.params.stopLoss)
                        self.order_stopLoss.addinfo(ordername="STOPLONG")
                        if self.params.verbose:
                            print('  STOP TRAILING')

            else:
                if not self.position:  # we left the market
                    self.broker.cancel(self.order_stopLoss)
                    self.order_stopLoss = None
                    if self.params.verbose:
                        print('SOLD @price: {:.2f} cost: {:.2f} comm: {:.2f} {}'.format(order.executed.price, order.executed.value, order.executed.comm, bt.num2date(order.executed.dt)))

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS {:.2f}, NET {:.2f}'.format(
            trade.pnl, trade.pnlcomm))


In [146]:

# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=1) # Multiply the stake by X

cerebro.broker.setcash(1000)

# if commission_val:
#     cerebro.broker.setcommission(commission=commission_val/100)




In [147]:
class PandasData(bt.feeds.PandasData):
    """
    Define the PandasData feed to work with the DataFrame columns.
    """
    columns = ('Open', 'High', 'Low', 'Close', 'Volume', 'OpenInterest')
# Convert the datetime column to a timestamp object
df['datetime'] = pd.to_datetime(df['Datetime'])


KeyError: 'Datetime'

In [148]:
df.set_index('Datetime', inplace=True)


KeyError: "None of ['Datetime'] are in the columns"

In [150]:
import backtrader as bt
from backtrader_plotting import Bokeh
from backtrader_plotting.schemes import Tradimo
# ======================================================================================================================
# MAIN
# ======================================================================================================================

# Create an instance of cerebro
cerebro = bt.Cerebro(stdstats=False)

# Be selective about what we chart
#cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.BuySell)
cerebro.addobserver(bt.observers.Value)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addobserver(bt.observers.Trades)

# Set the investment capital
cerebro.broker.setcash(2000)
cerebro.broker.setcommission(0.01)

# Set position size
# cerebro.addsizer(bt.sizers.PercentSizer, percents=100)

# Add our strategy
# cerebro.addstrategy(MyStrategy )#, rsi_period=14, ema1 = (20),ema2 = (100), ema3 = (200))
# cerebro.optstrategy(firstStrategy, rsi_period=14)
cerebro.addstrategy(firstStrategy, rsi_period=21, ema1 = (20),ema2 = (100), ema3 = (200))#, rsi_period=14, ema1 = (20),ema2 = (100), ema3 = (200))

# cerebro.addstrategy(RSIStrategy, maperiod=14, quantity=0.01, stopLoss=.01, upper=60, lower=25)


data = PandasData(dataname=df)
cerebro.adddata(data)


# Add analyzers
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='ta')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0, annualize=True, timeframe=bt.TimeFrame.Days)
cerebro.addanalyzer(bt.analyzers.VWR, _name='vwr')
cerebro.addanalyzer(bt.analyzers.SQN, _name='sqn')
cerebro.addanalyzer(bt.analyzers.Transactions, _name='txn')
# # Run our Backtest
backtest = cerebro.run()
backtest_results = backtest[0]



RSI Period: 21 EMA1: 20 EMA2 : 100 EMA3: 200 Final PnL: -16041530.75


ZeroDivisionError: float division by zero

In [133]:
figure = cerebro.plot(style ='candlebars')[0][0]
figure.savefig('example.png')

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/Users/rehman/.conda/envs/test_project/lib/python3.9/site-packages/matplotlib/dates.py)

In [127]:
printTradeAnalysis(cerebro,backtest_results.analyzers)

Backtesting Results


KeyError: 

In [12]:
# cerebro.optstrategy(firstStrategy,  rsi_period=14, ema1 = (10,20),ema2 = (50,100), ema3 = (200))
# opt_runs = cerebro.run(maxcpus = 1)

# # Generate results list
# final_results_list = []
# for run in opt_runs:
#     for strategy in run:
#         value = round(strategy.broker.get_value(),2)
#         PnL = round(value - startcash,2)
#         period = strategy.params.period
#         final_results_list.append([period,PnL])

# #Sort Results List
# by_period = sorted(final_results_list, key=lambda x: x[0])
# by_PnL = sorted(final_results_list, key=lambda x: x[1], reverse=True)

# #Print results
# print('Results: Ordered by period:')
# for result in by_period:
#     print('Period: {}, PnL: {}'.format(result[0], result[1]))
# print('Results: Ordered by Profit:')
# for result in by_PnL:
#     print('Period: {}, PnL: {}'.format(result[0], result[1]))